In [1]:
from lohrasb.best_estimator import BaseModel
import xgboost
from optuna.pruners import HyperbandPruner
from optuna.samplers._tpe.sampler import TPESampler
from sklearn.model_selection import KFold,train_test_split
import pandas as pd
import numpy as np
import optuna
from sklearn.pipeline import Pipeline
from feature_engine.imputation import (
    CategoricalImputer,
    MeanMedianImputer
    )
from category_encoders import OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score)
from sklearn.metrics import f1_score, mean_absolute_error,r2_score
from sklearn.linear_model import *
from sklearn.svm import *
from xgboost import *
from sklearn.linear_model import *
from catboost import *
from lightgbm import *
from sklearn.neural_network import *
from imblearn.ensemble import *
from sklearn.ensemble import *


/Users/hjavedani/Documents/Lohrasb/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/hjavedani/Documents/Lohrasb/.venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[I 2022-09-22 11:17:59,754] A new study created in memory with name: no-name-96c900b4-d976-444f-bd36-4b171e43a63c
[I 2022-09-22 11:17:59,755] A new study created in memory with name: no-name-946f6cc7-7619-40df-8047-7d5ed3280926


# Example 1 :Computer Hardware Data Set (a regression problem)
  
https://archive.ics.uci.edu/ml/datasets/Computer+Hardware

In [2]:
urldata= "https://archive.ics.uci.edu/ml/machine-learning-databases/cpu-performance/machine.data"
# column names
col_names=[
    "vendor name",
    "Model Name",
    "MYCT",
    "MMIN",
    "MMAX",
    "CACH",
    "CHMIN",
    "CHMAX",
    "PRP"
]
# read data
data = pd.read_csv(urldata,header=None,names=col_names,sep=',')
data

,vendor name,Model Name,MYCT,MMIN,MMAX,CACH,CHMIN,CHMAX,PRP
adviser,32/60,125,256,6000,256,16,128,198,199
amdahl,470v/7,29,8000,32000,32,8,32,269,253
amdahl,470v/7a,29,8000,32000,32,8,32,220,253
amdahl,470v/7b,29,8000,32000,32,8,32,172,253
amdahl,470v/7c,29,8000,16000,32,8,16,132,132
...,...,...,...,...,...,...,...,...,...
sperry,80/8,124,1000,8000,0,1,8,42,37
sperry,90/80-model-3,98,1000,8000,32,2,8,46,50
sratus,32,125,2000,8000,0,2,14,52,41
wang,vs-100,480,512,8000,32,0,0,67,47


# Train test split

In [3]:
X = data.loc[:, data.columns != "PRP"]
y = data.loc[:, data.columns == "PRP"]
y = y.values.ravel()


X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.33, random_state=42)


# Find feature types for later use

In [4]:
int_cols =  X_train.select_dtypes(include=['int']).columns.tolist()
float_cols =  X_train.select_dtypes(include=['float']).columns.tolist()
cat_cols =  X_train.select_dtypes(include=['object']).columns.tolist()


#  Define estimator and set its arguments  

In [5]:
estimator = LinearRegression()
estimator_params= {
        "fit_intercept": [True, False],
    }


In [6]:
obj = BaseModel.bestmodel_factory.using_optuna(
            estimator=estimator,
            estimator_params=estimator_params,
            measure_of_accuracy="r2",
            verbose=3,
            n_jobs=-1,
            random_state=42,
            # optuna params
            # optuna study init params
            study=optuna.create_study(
                storage=None,
                sampler=TPESampler(),
                pruner=HyperbandPruner(),
                study_name=None,
                direction="maximize",
                load_if_exists=False,
                directions=None,
            ),
            # optuna optimization params
            study_optimize_objective=None,
            study_optimize_objective_n_trials=10,
            study_optimize_objective_timeout=600,
            study_optimize_n_jobs=-1,
            study_optimize_catch=(),
            study_optimize_callbacks=None,
            study_optimize_gc_after_trial=False,
            study_optimize_show_progress_bar=False,
        )

[I 2022-09-22 11:18:00,311] A new study created in memory with name: no-name-2a1d9620-70d9-4b3b-b278-3a6b0da2ce44


# Build sklearn pipeline

In [7]:


pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # regression model 
            ('obj', obj),


 ])



# Run Pipeline

In [8]:
pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)


/Users/hjavedani/Documents/Lohrasb/.venv/lib/python3.10/site-packages/optuna/study/study.py:393: FutureWarning: `n_jobs` argument has been deprecated in v2.7.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.7.0.
  warnings.warn(
[I 2022-09-22 11:18:00,451] Trial 2 finished with value: 0.9561648243794818 and parameters: {'fit_intercept': 1}. Best is trial 2 with value: 0.9561648243794818.
[I 2022-09-22 11:18:00,452] Trial 3 finished with value: 0.9495445619978359 and parameters: {'fit_intercept': 0}. Best is trial 2 with value: 0.9561648243794818.
[I 2022-09-22 11:18:00,456] Trial 1 finished with value: 0.9495445619978359 and parameters: {'fit_intercept': 0}. Best is trial 2 with value: 0.9561648243794818.
[I 2022-09-22 11:18:00,463] Trial 4 finished with value: 0.9561648243794818 and parameters: {'fit_intercept': 1}. Best is trial 2 with value: 0.9561648243794818.
[I 2022-09-22 11:18:00,463] Trial 7 finished with value: 0.9561648243794818 

Initializing Optuna
{'fit_intercept': 1}
{'fit_intercept': 0}
{'fit_intercept': 1}
{'fit_intercept': 0}
{'fit_intercept': 1}
{'fit_intercept': 0}
{'fit_intercept': 1}
{'fit_intercept': 1}
{'fit_intercept': 0}
{'fit_intercept': 1}
LinearRegression(fit_intercept=1)
Initializing Optuna
{'fit_intercept': 1}
{'fit_intercept': 1}
{'fit_intercept': 1}
{'fit_intercept': 1}
{'fit_intercept': 1}
{'fit_intercept': 1}
{'fit_intercept': 1}
{'fit_intercept': 1}
{'fit_intercept': 0}
{'fit_intercept': 1}


# Check performance of the pipeline

In [9]:
print('r2 score : ')
print(r2_score(y_test,y_pred))


r2 score : 
0.9402415944774102
